In [3]:
import pandas as pd
import scipy as sc
import numpy as np
import sklearn
import pickle
import pathlib as Path
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sb
sb.set(font='IPAexGothic')

import multiprocessing
import itertools
import collections
import datetime
import gc

from tqdm._tqdm_notebook import tqdm

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 50)

data_path = Path.Path("../data")
result_path = Path.Path("../result")

prefix = 'ana101'

In [4]:
pd.set_option('display.max_rows', 100)

# データ抽出
> ・全体データ（ana005_all_df.pkl）

In [5]:
all_df = pd.read_pickle(data_path / "ana005_all_df.pkl")
all_df

,id,B,O,S,b1,b2,b3,ballPositionLabel,ballX,ballY,batter,batterHand,battingType,dir,dist,flg_train,gameID,inning,isOuts,pitchType,pitcher,pitcherHand,speed,totalPitchingCount,y,topTeam,bottomTeam,batterTeam,pitcherTeam,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4
0,0,0,0,0,False,False,False,内角低め,17.0,J,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,ストレート,今永 昇太,L,149km/h,1,0.0,広島,DeNA,広島,DeNA,False,False,False,False,False
1,1,1,0,0,False,False,False,内角低め,14.0,I,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,ストレート,今永 昇太,L,149km/h,2,1.0,広島,DeNA,広島,DeNA,False,False,False,False,False
2,2,1,0,1,False,False,False,外角高め,8.0,D,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,チェンジアップ,今永 昇太,L,137km/h,3,0.0,広島,DeNA,広島,DeNA,False,False,False,False,False
3,3,2,0,1,False,False,False,内角中心,21.0,G,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,スライダー,今永 昇太,L,138km/h,4,2.0,広島,DeNA,広島,DeNA,False,False,False,False,False
4,4,2,0,2,False,False,False,外角中心,7.0,F,ピレラ,R,G,S,38.3,1,20202173,1回表,False,チェンジアップ,今永 昇太,L,136km/h,5,4.0,広島,DeNA,広島,DeNA,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54203,33803,0,0,1,False,False,False,NaN,NaN,NaN,坂口 智隆,L,NaN,NaN,NaN,0,20202023,5回表,NaN,NaN,床田 寛樹,L,NaN,2,NaN,ヤクルト,広島,ヤクルト,広島,False,False,False,False,False
54204,33804,0,0,0,False,False,False,NaN,NaN,NaN,メヒア,R,NaN,NaN,NaN,0,20202640,9回表,NaN,NaN,堀岡 隼人,R,NaN,1,NaN,広島,巨人,広島,巨人,False,False,False,False,False
54205,33805,0,0,0,True,False,False,NaN,NaN,NaN,鈴木 誠也,R,NaN,NaN,NaN,0,20202864,7回裏,NaN,NaN,ディプラン,R,NaN,1,NaN,巨人,広島,広島,巨人,False,False,False,False,False
54206,33806,3,1,1,False,True,False,NaN,NaN,NaN,周東 佑京,L,NaN,NaN,NaN,0,20202806,8回裏,NaN,NaN,田村 伊知郎,R,NaN,5,NaN,西武,ソフトバンク,ソフトバンク,西武,False,False,False,False,False


# 必要な列定義
> ・id <br>
> ・バッター名（batter）<br>
> ・ヒット以上有無（is_hit）<br>
> ・試合ID, イニング（gameID, inning） <br>
> ・投手利き手（pitcherHand）<br>
> ・ピッチャーチーム名（pitcherTeam）<br>
> ・出塁状態（b1, b2, b3）<br>
> ・カウント（B, O, S）<br>

In [6]:
# 必要な列定義
cols = ['id', 'batter', 'batterTeam', 'gameID', 'inning', 'pitcherHand', 'pitcherTeam', 'b1', 'b2', 'b3', 
        'B', 'O', 'S',
        'is_hit0', 'is_hit1', 'is_hit2', 'is_hit3', 'is_hit4',
        'flg_train'
       ]

In [7]:
# 使用する列のみ抽出
_all_df = all_df[cols]
_all_df

,id,batter,batterTeam,gameID,inning,pitcherHand,pitcherTeam,b1,b2,b3,B,O,S,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4,flg_train
0,0,ピレラ,広島,20202173,1回表,L,DeNA,False,False,False,0,0,0,False,False,False,False,False,1
1,1,ピレラ,広島,20202173,1回表,L,DeNA,False,False,False,1,0,0,False,False,False,False,False,1
2,2,ピレラ,広島,20202173,1回表,L,DeNA,False,False,False,1,0,1,False,False,False,False,False,1
3,3,ピレラ,広島,20202173,1回表,L,DeNA,False,False,False,2,0,1,False,False,False,False,False,1
4,4,ピレラ,広島,20202173,1回表,L,DeNA,False,False,False,2,0,2,True,True,False,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54203,33803,坂口 智隆,ヤクルト,20202023,5回表,L,広島,False,False,False,0,0,1,False,False,False,False,False,0
54204,33804,メヒア,広島,20202640,9回表,R,巨人,False,False,False,0,0,0,False,False,False,False,False,0
54205,33805,鈴木 誠也,広島,20202864,7回裏,R,巨人,True,False,False,0,0,0,False,False,False,False,False,0
54206,33806,周東 佑京,ソフトバンク,20202806,8回裏,R,西武,False,True,False,3,1,1,False,False,False,False,False,0


In [8]:
# 訓練データ抽出
train_df = _all_df[_all_df['flg_train'] == 1].drop('flg_train', axis=1).copy()
train_df

,id,batter,batterTeam,gameID,inning,pitcherHand,pitcherTeam,b1,b2,b3,B,O,S,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4
0,0,ピレラ,広島,20202173,1回表,L,DeNA,False,False,False,0,0,0,False,False,False,False,False
1,1,ピレラ,広島,20202173,1回表,L,DeNA,False,False,False,1,0,0,False,False,False,False,False
2,2,ピレラ,広島,20202173,1回表,L,DeNA,False,False,False,1,0,1,False,False,False,False,False
3,3,ピレラ,広島,20202173,1回表,L,DeNA,False,False,False,2,0,1,False,False,False,False,False
4,4,ピレラ,広島,20202173,1回表,L,DeNA,False,False,False,2,0,2,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20395,20395,大田 泰示,日本ハム,20202118,9回裏,R,ソフトバンク,False,False,False,1,2,0,False,False,False,False,False
20396,20396,大田 泰示,日本ハム,20202118,9回裏,R,ソフトバンク,False,False,False,1,2,1,False,False,False,False,False
20397,20397,大田 泰示,日本ハム,20202118,9回裏,R,ソフトバンク,False,False,False,2,2,1,False,False,False,False,False
20398,20398,大田 泰示,日本ハム,20202118,9回裏,R,ソフトバンク,False,False,False,2,2,2,False,False,False,False,False


# いろいろな打率
> チーム打率 <br>
> ・打率（単純な打率）<br>
> ・右（左）ピッチャー打率 <br>
> ・対戦チーム打率 <br>
> ・チャンス（出塁状態）打率 <br>
> ・カウント別打率 <br>
>
> 以上をヒットの種類（5種類）で作成する

In [9]:
# 打率計算関数
def clc_batting_avg(indt, hit, cols, col):
    
    # 入力データ
    _tmp0 = indt.copy()
    
    # 指定した区分ごとに、ヒットか否かを集約
    _tmp1 = _tmp0.groupby(cols)[hit].max().to_frame().reset_index()
    
    # バッター単位に打率を計算
    return _tmp1.groupby(col)[hit].mean().to_frame()

# ヒットの種類を定義

In [10]:
cols_hit = ['is_hit'+str(i) for i in range(5)]
cols_hit

['is_hit0', 'is_hit1', 'is_hit2', 'is_hit3', 'is_hit4']

# 　

# 0．チーム打率
> ・batter, gameID, inning ⇒　一打席

In [11]:
# バッターチームごとに
_batter0 = pd.concat(
    
                [
    
                    clc_batting_avg(

                        indt=train_df,

                        hit=_col,

                        cols=['batterTeam', 'batter', 'gameID', 'inning'],

                        col=['batterTeam']

                    )

                    # ヒットの種類別
                    for _col in cols_hit

                ]
                # 横結合
                , axis=1
    
            ).reset_index()

_batter0

,batterTeam,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4
0,DeNA,0.269542,0.188679,0.053908,0.000000,0.026954
1,オリックス,0.191781,0.136986,0.035616,0.000000,0.019178
2,ソフトバンク,0.190981,0.119363,0.037135,0.005305,0.029178
3,ヤクルト,0.226006,0.154799,0.043344,0.000000,0.030960
4,ロッテ,0.207650,0.139344,0.038251,0.000000,0.030055
5,中日,0.258152,0.187500,0.048913,0.005435,0.016304
6,巨人,0.258065,0.169355,0.045699,0.005376,0.040323
7,広島,0.269122,0.184136,0.036827,0.005666,0.042493
8,日本ハム,0.187328,0.112948,0.044077,0.002755,0.027548
9,楽天,0.265625,0.177083,0.067708,0.000000,0.023438


### 全体データに結合

In [12]:
_FTR0 = _all_df[['id', 'batter', 'batterTeam']].merge(

            _batter0.rename(columns={'is_hit'+str(i) : 'FTR_bt_btingavg0'+str(i) for i in range(5)}),
            
            how='left',
            on='batterTeam'
        )

_FTR0

,id,batter,batterTeam,FTR_bt_btingavg00,FTR_bt_btingavg01,FTR_bt_btingavg02,FTR_bt_btingavg03,FTR_bt_btingavg04
0,0,ピレラ,広島,0.269122,0.184136,0.036827,0.005666,0.042493
1,1,ピレラ,広島,0.269122,0.184136,0.036827,0.005666,0.042493
2,2,ピレラ,広島,0.269122,0.184136,0.036827,0.005666,0.042493
3,3,ピレラ,広島,0.269122,0.184136,0.036827,0.005666,0.042493
4,4,ピレラ,広島,0.269122,0.184136,0.036827,0.005666,0.042493
...,...,...,...,...,...,...,...,...
54203,33803,坂口 智隆,ヤクルト,0.226006,0.154799,0.043344,0.000000,0.030960
54204,33804,メヒア,広島,0.269122,0.184136,0.036827,0.005666,0.042493
54205,33805,鈴木 誠也,広島,0.269122,0.184136,0.036827,0.005666,0.042493
54206,33806,周東 佑京,ソフトバンク,0.190981,0.119363,0.037135,0.005305,0.029178


In [13]:
_FTR0[['FTR_bt_btingavg0'+str(i) for i in range(5)]].isnull().sum()

FTR_bt_btingavg00    0
FTR_bt_btingavg01    0
FTR_bt_btingavg02    0
FTR_bt_btingavg03    0
FTR_bt_btingavg04    0
dtype: int64

☝：12球団すべて結合済み

# 　

## 1．単純な打率
> ・gameID, inning　⇒　一打席

In [14]:
# バッターごとに、一打席に集約
_batter1 = pd.concat(

                [
                    clc_batting_avg(

                        indt=train_df,
                        
                        hit=_col,

                        cols=['batter', 'gameID', 'inning'], 

                        col=['batter']

                    )

                    for _col in cols_hit

                ], axis=1
    
            ).reset_index()

_batter1.head(100)

,batter,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4
0,アルモンテ,0.242424,0.212121,0.000000,0.000000,0.030303
1,イノーア,0.000000,0.000000,0.000000,0.000000,0.000000
2,ウィーラー,0.333333,0.333333,0.000000,0.000000,0.000000
3,エスコバー,0.200000,0.166667,0.033333,0.000000,0.000000
4,オースティン,0.375000,0.218750,0.093750,0.000000,0.062500
5,ガルシア,0.000000,0.000000,0.000000,0.000000,0.000000
6,ガンケル,0.000000,0.000000,0.000000,0.000000,0.000000
7,サンズ,0.083333,0.000000,0.000000,0.000000,0.083333
8,サンチェス,0.000000,0.000000,0.000000,0.000000,0.000000
9,ジョーンズ,0.225000,0.150000,0.025000,0.000000,0.050000


### 全体データに結合

In [15]:
_FTR1 = _all_df[['id', 'batter']].merge(

            _batter1.rename(columns={'is_hit'+str(i) : 'FTR_bt_btingavg1'+str(i) for i in range(5)}),
            
            how='left',
            on='batter'
        )

_FTR1

,id,batter,FTR_bt_btingavg10,FTR_bt_btingavg11,FTR_bt_btingavg12,FTR_bt_btingavg13,FTR_bt_btingavg14
0,0,ピレラ,0.333333,0.238095,0.047619,0.000,0.047619
1,1,ピレラ,0.333333,0.238095,0.047619,0.000,0.047619
2,2,ピレラ,0.333333,0.238095,0.047619,0.000,0.047619
3,3,ピレラ,0.333333,0.238095,0.047619,0.000,0.047619
4,4,ピレラ,0.333333,0.238095,0.047619,0.000,0.047619
...,...,...,...,...,...,...,...
54203,33803,坂口 智隆,0.250000,0.200000,0.025000,0.000,0.025000
54204,33804,メヒア,0.121212,0.090909,0.000000,0.000,0.030303
54205,33805,鈴木 誠也,0.325000,0.150000,0.025000,0.025,0.125000
54206,33806,周東 佑京,0.000000,0.000000,0.000000,0.000,0.000000


In [16]:
_FTR1[['FTR_bt_btingavg1'+str(i) for i in range(5)]].isnull().sum()

FTR_bt_btingavg10    5348
FTR_bt_btingavg11    5348
FTR_bt_btingavg12    5348
FTR_bt_btingavg13    5348
FTR_bt_btingavg14    5348
dtype: int64

☝：訓練データにいないバッター（チーム打率で補完）

# 　

## 2．ピッチャーの利き手別打率
> ・gameID, inning, pitcherHand ⇒　投手の利き手別：一打席

In [17]:
_batter2 = pd.concat(
    
                [
                    clc_batting_avg(

                        # 入力データ：必要データのみ訓練データ
                        indt=train_df, 
                        
                        hit=_col,

                        # 集約単位：打席×投手の利き手
                        cols=['batter', 'gameID', 'inning', 'pitcherHand'], 

                        # 出力集約状態：バッター×投手の利き手
                        col=['batter', 'pitcherHand']

                    )
                    
                    for _col in cols_hit
                    
                ], axis=1
    
            ).reset_index()

_batter2.head(100)

,batter,pitcherHand,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4
0,アルモンテ,L,0.400000,0.300000,0.000000,0.000000,0.100000
1,アルモンテ,R,0.173913,0.173913,0.000000,0.000000,0.000000
2,イノーア,R,0.000000,0.000000,0.000000,0.000000,0.000000
3,ウィーラー,L,0.500000,0.500000,0.000000,0.000000,0.000000
4,ウィーラー,R,0.000000,0.000000,0.000000,0.000000,0.000000
5,エスコバー,L,0.222222,0.111111,0.111111,0.000000,0.000000
6,エスコバー,R,0.190476,0.190476,0.000000,0.000000,0.000000
7,オースティン,L,0.285714,0.142857,0.142857,0.000000,0.000000
8,オースティン,R,0.400000,0.240000,0.080000,0.000000,0.080000
9,ガルシア,R,0.000000,0.000000,0.000000,0.000000,0.000000


☝：対戦する投手の利き手ごとに結合するので、バッター単位には集約しない

### 全体データに結合

In [18]:
_FTR2 = _all_df[['id', 'batter', 'pitcherHand']].merge(

            _batter2.rename(columns={'is_hit'+str(i) : 'FTR_bt_btingavg2' + str(i) for i in range(5)}),
            
            how='left',
    
            on=['batter', 'pitcherHand']
    
        )

_FTR2

,id,batter,pitcherHand,FTR_bt_btingavg20,FTR_bt_btingavg21,FTR_bt_btingavg22,FTR_bt_btingavg23,FTR_bt_btingavg24
0,0,ピレラ,L,0.500000,0.500000,0.000000,0.0,0.000000
1,1,ピレラ,L,0.500000,0.500000,0.000000,0.0,0.000000
2,2,ピレラ,L,0.500000,0.500000,0.000000,0.0,0.000000
3,3,ピレラ,L,0.500000,0.500000,0.000000,0.0,0.000000
4,4,ピレラ,L,0.500000,0.500000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...
54203,33803,坂口 智隆,L,0.375000,0.250000,0.125000,0.0,0.000000
54204,33804,メヒア,R,0.086957,0.086957,0.000000,0.0,0.000000
54205,33805,鈴木 誠也,R,0.285714,0.178571,0.035714,0.0,0.071429
54206,33806,周東 佑京,R,0.000000,0.000000,0.000000,0.0,0.000000


In [19]:
_FTR2[['FTR_bt_btingavg2'+str(i) for i in range(5)]].isnull().sum()

FTR_bt_btingavg20    6053
FTR_bt_btingavg21    6053
FTR_bt_btingavg22    6053
FTR_bt_btingavg23    6053
FTR_bt_btingavg24    6053
dtype: int64

☝：対戦したことの無い利き手（バッター打率で補完）

# 　

## 3．対戦チーム打率
> ・gameID, inning, pitcherTeam　⇒　投手のチーム別：一打席

In [20]:
_batter3 = pd.concat(
    
                [
                    clc_batting_avg(

                        indt = train_df,

                        hit=_col,

                        cols=['batter', 'gameID', 'inning', 'pitcherTeam'],

                        col=['batter', 'pitcherTeam']

                    )
                    
                    for _col in cols_hit
                    
                ], axis=1
            
            ).reset_index()

_batter3.head(100)

,batter,pitcherTeam,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4
0,アルモンテ,DeNA,0.100000,0.100000,0.000000,0.000000,0.000000
1,アルモンテ,ヤクルト,0.400000,0.400000,0.000000,0.000000,0.000000
2,アルモンテ,広島,0.222222,0.222222,0.000000,0.000000,0.000000
3,アルモンテ,阪神,0.250000,0.000000,0.000000,0.000000,0.250000
4,イノーア,阪神,0.000000,0.000000,0.000000,0.000000,0.000000
5,ウィーラー,DeNA,0.333333,0.333333,0.000000,0.000000,0.000000
6,エスコバー,中日,0.272727,0.181818,0.090909,0.000000,0.000000
7,エスコバー,巨人,0.100000,0.100000,0.000000,0.000000,0.000000
8,エスコバー,阪神,0.222222,0.222222,0.000000,0.000000,0.000000
9,オースティン,中日,0.500000,0.416667,0.000000,0.000000,0.083333


☝：対戦チームごとに結合するので、バッター単位には集約しない

### 全体データに結合

In [21]:
_FTR3 = _all_df[['id', 'batter', 'pitcherTeam']].merge(

            _batter3.rename(columns={'is_hit'+str(i) : 'FTR_bt_btingavg3'+str(i) for i in range(5)}),
            
            how='left',
    
            on=['batter', 'pitcherTeam']
    
        )

_FTR3

,id,batter,pitcherTeam,FTR_bt_btingavg30,FTR_bt_btingavg31,FTR_bt_btingavg32,FTR_bt_btingavg33,FTR_bt_btingavg34
0,0,ピレラ,DeNA,0.357143,0.214286,0.071429,0.0,0.071429
1,1,ピレラ,DeNA,0.357143,0.214286,0.071429,0.0,0.071429
2,2,ピレラ,DeNA,0.357143,0.214286,0.071429,0.0,0.071429
3,3,ピレラ,DeNA,0.357143,0.214286,0.071429,0.0,0.071429
4,4,ピレラ,DeNA,0.357143,0.214286,0.071429,0.0,0.071429
...,...,...,...,...,...,...,...,...
54203,33803,坂口 智隆,広島,NaN,NaN,NaN,NaN,NaN
54204,33804,メヒア,巨人,0.083333,0.083333,0.000000,0.0,0.000000
54205,33805,鈴木 誠也,巨人,0.307692,0.153846,0.076923,0.0,0.076923
54206,33806,周東 佑京,西武,NaN,NaN,NaN,NaN,NaN


In [22]:
_FTR3[['FTR_bt_btingavg3'+str(i) for i in range(5)]].isnull().sum()

FTR_bt_btingavg30    17844
FTR_bt_btingavg31    17844
FTR_bt_btingavg32    17844
FTR_bt_btingavg33    17844
FTR_bt_btingavg34    17844
dtype: int64

☝：訓練データで対戦したことのないチーム（バッター打率で補完）

# 　

## 4．出塁状態ごとの打率
> ・gameID, inning, b1, b2, b3　⇒　出塁状態別：一打席

In [23]:
_batter4 = pd.concat(
    
                [
                    clc_batting_avg(

                        indt = train_df,
                        
                        hit = _col,

                        cols = ['batter', 'gameID', 'inning', 'b1', 'b2', 'b3'],

                        col = ['batter', 'b1', 'b2', 'b3']

                    )
                    
                    for _col in cols_hit
                    
                ], axis=1
    
            ).reset_index()

_batter4.head(100)

,batter,b1,b2,b3,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4
0,アルモンテ,False,False,False,0.190476,0.190476,0.000000,0.000000,0.000000
1,アルモンテ,False,True,False,0.500000,0.500000,0.000000,0.000000,0.000000
2,アルモンテ,True,False,False,0.333333,0.333333,0.000000,0.000000,0.000000
3,アルモンテ,True,True,False,0.000000,0.000000,0.000000,0.000000,0.000000
4,アルモンテ,True,True,True,1.000000,0.000000,0.000000,0.000000,1.000000
5,イノーア,False,False,False,0.000000,0.000000,0.000000,0.000000,0.000000
6,ウィーラー,False,False,False,0.500000,0.500000,0.000000,0.000000,0.000000
7,ウィーラー,True,True,False,0.000000,0.000000,0.000000,0.000000,0.000000
8,エスコバー,False,False,False,0.222222,0.222222,0.000000,0.000000,0.000000
9,エスコバー,False,False,True,0.000000,0.000000,0.000000,0.000000,0.000000


### 全体データに結合

In [24]:
_FTR4 = _all_df[['id', 'batter', 'b1', 'b2', 'b3']].merge(

            _batter4.rename(columns={'is_hit'+str(i) : 'FTR_bt_btingavg4'+str(i) for i in range(5)}),
            
            how='left',
    
            on=['batter', 'b1', 'b2', 'b3']
    
        )

_FTR4

,id,batter,b1,b2,b3,FTR_bt_btingavg40,FTR_bt_btingavg41,FTR_bt_btingavg42,FTR_bt_btingavg43,FTR_bt_btingavg44
0,0,ピレラ,False,False,False,0.275862,0.172414,0.034483,0.0,0.068966
1,1,ピレラ,False,False,False,0.275862,0.172414,0.034483,0.0,0.068966
2,2,ピレラ,False,False,False,0.275862,0.172414,0.034483,0.0,0.068966
3,3,ピレラ,False,False,False,0.275862,0.172414,0.034483,0.0,0.068966
4,4,ピレラ,False,False,False,0.275862,0.172414,0.034483,0.0,0.068966
...,...,...,...,...,...,...,...,...,...,...
54203,33803,坂口 智隆,False,False,False,0.285714,0.214286,0.035714,0.0,0.035714
54204,33804,メヒア,False,False,False,0.176471,0.117647,0.000000,0.0,0.058824
54205,33805,鈴木 誠也,True,False,False,0.384615,0.230769,0.076923,0.0,0.076923
54206,33806,周東 佑京,False,True,False,NaN,NaN,NaN,NaN,NaN


In [25]:
_FTR4[['FTR_bt_btingavg4'+str(i) for i in range(5)]].isnull().sum()

FTR_bt_btingavg40    7842
FTR_bt_btingavg41    7842
FTR_bt_btingavg42    7842
FTR_bt_btingavg43    7842
FTR_bt_btingavg44    7842
dtype: int64

☝：訓練データで発生していない出塁状態（バッター打率で補完）

# 　

# 5．カウント別打率
> ・gameID, inning, B, O, S　⇒　カウント状態別：一打席

In [26]:
_batter5 = pd.concat(
    
                [
                    clc_batting_avg(

                        indt = train_df,
                        
                        hit = _col,

                        cols = ['batter', 'gameID', 'inning', 'B', 'O', 'S'],

                        col = ['batter', 'B', 'O', 'S']

                    )
                    
                    for _col in cols_hit
                    
                ], axis=1
    
            ).reset_index()

_batter5.head(100)

,batter,B,O,S,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4
0,アルモンテ,0,0,0,0.083333,0.083333,0.000000,0.0,0.000000
1,アルモンテ,0,0,1,0.000000,0.000000,0.000000,0.0,0.000000
2,アルモンテ,0,0,2,0.000000,0.000000,0.000000,0.0,0.000000
3,アルモンテ,0,1,0,0.090909,0.000000,0.000000,0.0,0.090909
4,アルモンテ,0,1,1,0.000000,0.000000,0.000000,0.0,0.000000
5,アルモンテ,0,1,2,0.000000,0.000000,0.000000,0.0,0.000000
6,アルモンテ,0,2,0,0.000000,0.000000,0.000000,0.0,0.000000
7,アルモンテ,0,2,1,0.200000,0.200000,0.000000,0.0,0.000000
8,アルモンテ,0,2,2,0.000000,0.000000,0.000000,0.0,0.000000
9,アルモンテ,1,0,0,0.000000,0.000000,0.000000,0.0,0.000000


## 全体データに結合

In [27]:
_FTR5 = _all_df[['id', 'batter', 'B', 'O', 'S']].merge(

            _batter5.rename(columns={'is_hit'+str(i) : 'FTR_bt_btingavg5'+str(i) for i in range(5)}),
            
            how='left',
    
            on=['batter', 'B', 'O', 'S']
    
        )

_FTR5

,id,batter,B,O,S,FTR_bt_btingavg50,FTR_bt_btingavg51,FTR_bt_btingavg52,FTR_bt_btingavg53,FTR_bt_btingavg54
0,0,ピレラ,0,0,0,0.000000,0.000000,0.0,0.0,0.000000
1,1,ピレラ,1,0,0,0.000000,0.000000,0.0,0.0,0.000000
2,2,ピレラ,1,0,1,0.000000,0.000000,0.0,0.0,0.000000
3,3,ピレラ,2,0,1,0.000000,0.000000,0.0,0.0,0.000000
4,4,ピレラ,2,0,2,0.333333,0.166667,0.0,0.0,0.166667
...,...,...,...,...,...,...,...,...,...,...
54203,33803,坂口 智隆,0,0,1,0.125000,0.125000,0.0,0.0,0.000000
54204,33804,メヒア,0,0,0,0.000000,0.000000,0.0,0.0,0.000000
54205,33805,鈴木 誠也,0,0,0,0.000000,0.000000,0.0,0.0,0.000000
54206,33806,周東 佑京,3,1,1,NaN,NaN,NaN,NaN,NaN


In [28]:
_FTR5[['FTR_bt_btingavg5'+str(i) for i in range(5)]].isnull().sum()

FTR_bt_btingavg50    9285
FTR_bt_btingavg51    9285
FTR_bt_btingavg52    9285
FTR_bt_btingavg53    9285
FTR_bt_btingavg54    9285
dtype: int64

☝：訓練データで遭遇していないカウントの打席（単純な打率で補完）

# 　

# データ結合
> ・作成した状況別打率を、全体データ（_all_df）に結合する

In [37]:
_FTR0.head()

,id,batter,batterTeam,FTR_bt_btingavg00,FTR_bt_btingavg01,FTR_bt_btingavg02,FTR_bt_btingavg03,FTR_bt_btingavg04
0,0,ピレラ,広島,0.269122,0.184136,0.036827,0.005666,0.042493
1,1,ピレラ,広島,0.269122,0.184136,0.036827,0.005666,0.042493
2,2,ピレラ,広島,0.269122,0.184136,0.036827,0.005666,0.042493
3,3,ピレラ,広島,0.269122,0.184136,0.036827,0.005666,0.042493
4,4,ピレラ,広島,0.269122,0.184136,0.036827,0.005666,0.042493


In [38]:
_FTR1.head()

,id,batter,FTR_bt_btingavg10,FTR_bt_btingavg11,FTR_bt_btingavg12,FTR_bt_btingavg13,FTR_bt_btingavg14
0,0,ピレラ,0.333333,0.238095,0.047619,0.0,0.047619
1,1,ピレラ,0.333333,0.238095,0.047619,0.0,0.047619
2,2,ピレラ,0.333333,0.238095,0.047619,0.0,0.047619
3,3,ピレラ,0.333333,0.238095,0.047619,0.0,0.047619
4,4,ピレラ,0.333333,0.238095,0.047619,0.0,0.047619


In [39]:
_FTR2.head()

,id,batter,pitcherHand,FTR_bt_btingavg20,FTR_bt_btingavg21,FTR_bt_btingavg22,FTR_bt_btingavg23,FTR_bt_btingavg24
0,0,ピレラ,L,0.5,0.5,0.0,0.0,0.0
1,1,ピレラ,L,0.5,0.5,0.0,0.0,0.0
2,2,ピレラ,L,0.5,0.5,0.0,0.0,0.0
3,3,ピレラ,L,0.5,0.5,0.0,0.0,0.0
4,4,ピレラ,L,0.5,0.5,0.0,0.0,0.0


In [40]:
_FTR3.head()

,id,batter,pitcherTeam,FTR_bt_btingavg30,FTR_bt_btingavg31,FTR_bt_btingavg32,FTR_bt_btingavg33,FTR_bt_btingavg34
0,0,ピレラ,DeNA,0.357143,0.214286,0.071429,0.0,0.071429
1,1,ピレラ,DeNA,0.357143,0.214286,0.071429,0.0,0.071429
2,2,ピレラ,DeNA,0.357143,0.214286,0.071429,0.0,0.071429
3,3,ピレラ,DeNA,0.357143,0.214286,0.071429,0.0,0.071429
4,4,ピレラ,DeNA,0.357143,0.214286,0.071429,0.0,0.071429


In [41]:
_FTR4.head()

,id,batter,b1,b2,b3,FTR_bt_btingavg40,FTR_bt_btingavg41,FTR_bt_btingavg42,FTR_bt_btingavg43,FTR_bt_btingavg44
0,0,ピレラ,False,False,False,0.275862,0.172414,0.034483,0.0,0.068966
1,1,ピレラ,False,False,False,0.275862,0.172414,0.034483,0.0,0.068966
2,2,ピレラ,False,False,False,0.275862,0.172414,0.034483,0.0,0.068966
3,3,ピレラ,False,False,False,0.275862,0.172414,0.034483,0.0,0.068966
4,4,ピレラ,False,False,False,0.275862,0.172414,0.034483,0.0,0.068966


In [29]:
_FTR5.head()

,id,batter,B,O,S,FTR_bt_btingavg50,FTR_bt_btingavg51,FTR_bt_btingavg52,FTR_bt_btingavg53,FTR_bt_btingavg54
0,0,ピレラ,0,0,0,0.000000,0.000000,0.0,0.0,0.000000
1,1,ピレラ,1,0,0,0.000000,0.000000,0.0,0.0,0.000000
2,2,ピレラ,1,0,1,0.000000,0.000000,0.0,0.0,0.000000
3,3,ピレラ,2,0,1,0.000000,0.000000,0.0,0.0,0.000000
4,4,ピレラ,2,0,2,0.333333,0.166667,0.0,0.0,0.166667


In [30]:
_FTR_btingavg = pd.concat(
    
                    [
                        _all_df,
                        
                        # 特徴量のみ抽出
                        _FTR0.filter(like='FTR', axis=1), 
                        _FTR1.filter(like='FTR', axis=1),
                        _FTR2.filter(like='FTR', axis=1),
                        _FTR3.filter(like='FTR', axis=1),
                        _FTR4.filter(like='FTR', axis=1),
                        _FTR5.filter(like='FTR', axis=1)
                    ], 
                    axis=1

            )

_FTR_btingavg.head(100)

,id,batter,batterTeam,gameID,inning,pitcherHand,pitcherTeam,b1,b2,b3,B,O,S,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4,flg_train,FTR_bt_btingavg00,FTR_bt_btingavg01,FTR_bt_btingavg02,FTR_bt_btingavg03,FTR_bt_btingavg04,FTR_bt_btingavg10,FTR_bt_btingavg11,FTR_bt_btingavg12,FTR_bt_btingavg13,FTR_bt_btingavg14,FTR_bt_btingavg20,FTR_bt_btingavg21,FTR_bt_btingavg22,FTR_bt_btingavg23,FTR_bt_btingavg24,FTR_bt_btingavg30,FTR_bt_btingavg31,FTR_bt_btingavg32,FTR_bt_btingavg33,FTR_bt_btingavg34,FTR_bt_btingavg40,FTR_bt_btingavg41,FTR_bt_btingavg42,FTR_bt_btingavg43,FTR_bt_btingavg44,FTR_bt_btingavg50,FTR_bt_btingavg51,FTR_bt_btingavg52,FTR_bt_btingavg53,FTR_bt_btingavg54
0,0,ピレラ,広島,20202173,1回表,L,DeNA,False,False,False,0,0,0,False,False,False,False,False,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.333333,0.238095,0.047619,0.000000,0.047619,0.500000,0.500000,0.000000,0.000000,0.000000,0.357143,0.214286,0.071429,0.000000,0.071429,0.275862,0.172414,0.034483,0.000000,0.068966,0.000000,0.000000,0.0,0.0,0.000000
1,1,ピレラ,広島,20202173,1回表,L,DeNA,False,False,False,1,0,0,False,False,False,False,False,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.333333,0.238095,0.047619,0.000000,0.047619,0.500000,0.500000,0.000000,0.000000,0.000000,0.357143,0.214286,0.071429,0.000000,0.071429,0.275862,0.172414,0.034483,0.000000,0.068966,0.000000,0.000000,0.0,0.0,0.000000
2,2,ピレラ,広島,20202173,1回表,L,DeNA,False,False,False,1,0,1,False,False,False,False,False,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.333333,0.238095,0.047619,0.000000,0.047619,0.500000,0.500000,0.000000,0.000000,0.000000,0.357143,0.214286,0.071429,0.000000,0.071429,0.275862,0.172414,0.034483,0.000000,0.068966,0.000000,0.000000,0.0,0.0,0.000000
3,3,ピレラ,広島,20202173,1回表,L,DeNA,False,False,False,2,0,1,False,False,False,False,False,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.333333,0.238095,0.047619,0.000000,0.047619,0.500000,0.500000,0.000000,0.000000,0.000000,0.357143,0.214286,0.071429,0.000000,0.071429,0.275862,0.172414,0.034483,0.000000,0.068966,0.000000,0.000000,0.0,0.0,0.000000
4,4,ピレラ,広島,20202173,1回表,L,DeNA,False,False,False,2,0,2,True,True,False,False,False,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.333333,0.238095,0.047619,0.000000,0.047619,0.500000,0.500000,0.000000,0.000000,0.000000,0.357143,0.214286,0.071429,0.000000,0.071429,0.275862,0.172414,0.034483,0.000000,0.068966,0.333333,0.166667,0.0,0.0,0.166667
5,5,菊池 涼介,広島,20202173,1回表,L,DeNA,True,False,False,0,0,0,False,False,False,False,False,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.214286,0.166667,0.023810,0.000000,0.023810,0.250000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
6,6,菊池 涼介,広島,20202173,1回表,L,DeNA,True,False,False,0,0,1,False,False,False,False,False,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.214286,0.166667,0.023810,0.000000,0.023810,0.250000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
7,7,西川 龍馬,広島,20202173,1回表,L,DeNA,True,False,False,0,1,0,False,False,False,False,False,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.292683,0.195122,0.073171,0.000000,0.024390,0.153846,0.000000,0.153846,0.000000,0.000000,0.230769,0.076923,0.153846,0.000000,0.000000,0.125000,0.125000,0.000000,0.000000,0.000000,0.125000,0.125000,0.0,0.0,0.000000
8,8,西川 龍馬,広島,20202173,1回表,L,DeNA,True,False,False,0,1,1,False,False,False,False,False,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.292683,0.195122,0.073171,0.000000,0.024390,0.153846,0.000000,0.153846,0.000000,0.000000,0.230769,0.076923,0.153846,0.000000,0.000000,0.125000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
9,9,西川 龍馬,広島,20202173,1回表,L,DeNA,True,False,False,0,1,2,False,False,False,False,False,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.292683,0.195122,0.073171,0.000000,0.024390,0.153846,0.000000,0.153846,0.000

# 　

# 欠損値処置
> ・バッター打率：チーム打率 <br>
> ・投手利き手別打率：バッター打率（補完済み）<br>
> ・対戦チーム打率：バッター打率（〃）<br>
> ・出塁状況別打率：バッター打率（〃）<br>
> ・カウント別打率：バッター打率（〃）<br>

In [31]:
# 処理後データ作成
FTR_btingavg = _FTR_btingavg.copy()

In [32]:
# 1．バッター打率：チーム打率
for i in range(5):

    FTR_btingavg['FTR_bt_btingavg1'+str(i)] = FTR_btingavg['FTR_bt_btingavg1'+str(i)].fillna(FTR_btingavg['FTR_bt_btingavg0'+str(i)])

In [33]:
for i in range(5):
    
    # 2．投手利き手別打率：バッター打率（補完済み）
    FTR_btingavg['FTR_bt_btingavg2'+str(i)] = FTR_btingavg['FTR_bt_btingavg2'+str(i)].fillna(FTR_btingavg['FTR_bt_btingavg1'+str(i)])
    
    # 3．対戦チーム打率：バッター打率（〃）
    FTR_btingavg['FTR_bt_btingavg3'+str(i)] = FTR_btingavg['FTR_bt_btingavg3'+str(i)].fillna(FTR_btingavg['FTR_bt_btingavg1'+str(i)])
    
    # 4．出塁状況別打率：バッター打率（〃）
    FTR_btingavg['FTR_bt_btingavg4'+str(i)] = FTR_btingavg['FTR_bt_btingavg4'+str(i)].fillna(FTR_btingavg['FTR_bt_btingavg1'+str(i)])
    
    # 5．カウント別打率：バッター打率（〃）
    FTR_btingavg['FTR_bt_btingavg5'+str(i)] = FTR_btingavg['FTR_bt_btingavg5'+str(i)].fillna(FTR_btingavg['FTR_bt_btingavg1'+str(i)])
    

補完処理の確認

In [34]:
FTR_btingavg[_FTR1['FTR_bt_btingavg10'].isnull()]

,id,batter,batterTeam,gameID,inning,pitcherHand,pitcherTeam,b1,b2,b3,B,O,S,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4,flg_train,FTR_bt_btingavg00,FTR_bt_btingavg01,FTR_bt_btingavg02,FTR_bt_btingavg03,FTR_bt_btingavg04,FTR_bt_btingavg10,FTR_bt_btingavg11,FTR_bt_btingavg12,FTR_bt_btingavg13,FTR_bt_btingavg14,FTR_bt_btingavg20,FTR_bt_btingavg21,FTR_bt_btingavg22,FTR_bt_btingavg23,FTR_bt_btingavg24,FTR_bt_btingavg30,FTR_bt_btingavg31,FTR_bt_btingavg32,FTR_bt_btingavg33,FTR_bt_btingavg34,FTR_bt_btingavg40,FTR_bt_btingavg41,FTR_bt_btingavg42,FTR_bt_btingavg43,FTR_bt_btingavg44,FTR_bt_btingavg50,FTR_bt_btingavg51,FTR_bt_btingavg52,FTR_bt_btingavg53,FTR_bt_btingavg54
20408,8,釜元 豪,ソフトバンク,20202068,5回裏,L,楽天,False,False,False,1,2,0,False,False,False,False,False,0,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178
20410,10,加藤 翔平,ロッテ,20202536,5回裏,R,西武,False,False,False,0,2,0,False,False,False,False,False,0,0.207650,0.139344,0.038251,0.000000,0.030055,0.207650,0.139344,0.038251,0.000000,0.030055,0.207650,0.139344,0.038251,0.000000,0.030055,0.207650,0.139344,0.038251,0.000000,0.030055,0.207650,0.139344,0.038251,0.000000,0.030055,0.207650,0.139344,0.038251,0.000000,0.030055
20411,11,太田 椋,オリックス,20202351,4回表,R,楽天,False,False,False,0,1,2,False,False,False,False,False,0,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178
20423,23,西野 真弘,オリックス,20202829,8回裏,R,日本ハム,False,False,False,2,1,0,False,False,False,False,False,0,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178
20427,27,川端 慎吾,ヤクルト,20202534,8回表,L,阪神,False,False,False,1,0,0,False,False,False,False,False,0,0.226006,0.154799,0.043344,0.000000,0.030960,0.226006,0.154799,0.043344,0.000000,0.030960,0.226006,0.154799,0.043344,0.000000,0.030960,0.226006,0.154799,0.043344,0.000000,0.030960,0.226006,0.154799,0.043344,0.000000,0.030960,0.226006,0.154799,0.043344,0.000000,0.030960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54176,33776,石川 昂弥,中日,20202350,6回裏,R,阪神,False,False,True,0,2,1,False,False,False,False,False,0,0.258152,0.187500,0.048913,0.005435,0.016304,0.258152,0.187500,0.048913,0.005435,0.016304,0.258152,0.187500,0.048913,0.005435,0.016304,0.258152,0.187500,0.048913,0.005435,0.016304,0.258152,0.187500,0.048913,0.005435,0.016304,0.258152,0.187500,0.048913,0.005435,0.016304
54183,33783,中村 晃,ソフトバンク,20202648,9回裏,R,オリックス,False,False,False,1,1,1,False,False,False,False,False,0,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178
54188,33788,山下 幸輝,DeNA,20202538,2回表,R,巨人,False,False,False,2,1,1,False,False,False,False,False,0,0.269542,0.188679,0.053908,0.000000,0.026954,0.269542,0.188679,0.053908,0.000000,0.026954,0.269542,0.188679,0.053908,0.000000,0.026954,0.269542,0.188679,0.053908,0.000000,0.026954,0.269542,0.188679,0.053908,0.000000,0.026954,0.269542,0.188679,0.053908,0.000000,0.026954
54189,33789,中村 晃,ソフトバンク,20202561,3回裏,R,ロッテ,False,False,False,2,2,1,False,False,False,False,False,0,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.03

In [35]:
FTR_btingavg[_FTR2['FTR_bt_btingavg20'].isnull()]

,id,batter,batterTeam,gameID,inning,pitcherHand,pitcherTeam,b1,b2,b3,B,O,S,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4,flg_train,FTR_bt_btingavg00,FTR_bt_btingavg01,FTR_bt_btingavg02,FTR_bt_btingavg03,FTR_bt_btingavg04,FTR_bt_btingavg10,FTR_bt_btingavg11,FTR_bt_btingavg12,FTR_bt_btingavg13,FTR_bt_btingavg14,FTR_bt_btingavg20,FTR_bt_btingavg21,FTR_bt_btingavg22,FTR_bt_btingavg23,FTR_bt_btingavg24,FTR_bt_btingavg30,FTR_bt_btingavg31,FTR_bt_btingavg32,FTR_bt_btingavg33,FTR_bt_btingavg34,FTR_bt_btingavg40,FTR_bt_btingavg41,FTR_bt_btingavg42,FTR_bt_btingavg43,FTR_bt_btingavg44,FTR_bt_btingavg50,FTR_bt_btingavg51,FTR_bt_btingavg52,FTR_bt_btingavg53,FTR_bt_btingavg54
986,986,角中 勝也,ロッテ,20202170,3回表,NaN,ソフトバンク,True,False,True,0,2,0,False,False,False,False,False,1,0.207650,0.139344,0.038251,0.000000,0.030055,0.138889,0.138889,0.000000,0.000000,0.000000,0.138889,0.138889,0.000000,0.000000,0.000000,0.133333,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
987,987,角中 勝也,ロッテ,20202170,3回表,NaN,ソフトバンク,True,False,True,0,2,1,False,False,False,False,False,1,0.207650,0.139344,0.038251,0.000000,0.030055,0.138889,0.138889,0.000000,0.000000,0.000000,0.138889,0.138889,0.000000,0.000000,0.000000,0.133333,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
988,988,角中 勝也,ロッテ,20202170,3回表,NaN,ソフトバンク,True,False,True,1,2,1,False,False,False,False,False,1,0.207650,0.139344,0.038251,0.000000,0.030055,0.138889,0.138889,0.000000,0.000000,0.000000,0.138889,0.138889,0.000000,0.000000,0.000000,0.133333,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2975,2975,大城 滉二,オリックス,20202165,1回裏,NaN,楽天,True,False,True,0,2,0,False,False,False,False,False,1,0.191781,0.136986,0.035616,0.000000,0.019178,0.028571,0.028571,0.000000,0.000000,0.000000,0.028571,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2976,2976,大城 滉二,オリックス,20202165,1回裏,NaN,楽天,True,False,True,1,2,0,False,False,False,False,False,1,0.191781,0.136986,0.035616,0.000000,0.019178,0.028571,0.028571,0.000000,0.000000,0.000000,0.028571,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54186,33786,西田 明央,ヤクルト,20202736,9回表,L,広島,False,False,False,1,2,2,False,False,False,False,False,0,0.226006,0.154799,0.043344,0.000000,0.030960,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
54188,33788,山下 幸輝,DeNA,20202538,2回表,R,巨人,False,False,False,2,1,1,False,False,False,False,False,0,0.269542,0.188679,0.053908,0.000000,0.026954,0.269542,0.188679,0.053908,0.000000,0.026954,0.269542,0.188679,0.053908,0.000000,0.026954,0.269542,0.188679,0.053908,0.000000,0.026954,0.269542,0.188679,0.053908,0.000000,0.026954,0.269542,0.188679,0.053908,0.000000,0.026954
54189,33789,中村 晃,ソフトバンク,20202561,3回裏,R,ロッテ,False,False,False,2,2,1,False,False,False,False,False,0,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178
54195,33795,西田 明央,ヤクルト,20202023,2回表,L,広島,False,True,False,1,0,2,False,False,False,False,False,0,0.226006,0.154799,0.043344,0.000000,0.030960,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [36]:
FTR_btingavg[_FTR3['FTR_bt_btingavg30'].isnull()]

,id,batter,batterTeam,gameID,inning,pitcherHand,pitcherTeam,b1,b2,b3,B,O,S,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4,flg_train,FTR_bt_btingavg00,FTR_bt_btingavg01,FTR_bt_btingavg02,FTR_bt_btingavg03,FTR_bt_btingavg04,FTR_bt_btingavg10,FTR_bt_btingavg11,FTR_bt_btingavg12,FTR_bt_btingavg13,FTR_bt_btingavg14,FTR_bt_btingavg20,FTR_bt_btingavg21,FTR_bt_btingavg22,FTR_bt_btingavg23,FTR_bt_btingavg24,FTR_bt_btingavg30,FTR_bt_btingavg31,FTR_bt_btingavg32,FTR_bt_btingavg33,FTR_bt_btingavg34,FTR_bt_btingavg40,FTR_bt_btingavg41,FTR_bt_btingavg42,FTR_bt_btingavg43,FTR_bt_btingavg44,FTR_bt_btingavg50,FTR_bt_btingavg51,FTR_bt_btingavg52,FTR_bt_btingavg53,FTR_bt_btingavg54
20402,2,堂林 翔太,広島,20203305,9回裏,R,阪神,True,False,False,3,2,2,False,False,False,False,False,0,0.269122,0.184136,0.036827,0.005666,0.042493,0.400000,0.266667,0.066667,0.000000,0.066667,0.476190,0.333333,0.047619,0.000000,0.095238,0.400000,0.266667,0.066667,0.000000,0.066667,0.571429,0.428571,0.000000,0.000000,0.142857,0.333333,0.000000,0.000000,0.000000,0.333333
20406,6,金子 侑司,西武,20202800,3回裏,L,ロッテ,False,False,False,0,0,1,False,False,False,False,False,0,0.206434,0.147453,0.032172,0.002681,0.024129,0.162162,0.135135,0.027027,0.000000,0.000000,0.153846,0.153846,0.000000,0.000000,0.000000,0.162162,0.135135,0.027027,0.000000,0.000000,0.058824,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
20407,7,栗原 陵矢,ソフトバンク,20202680,6回表,R,楽天,False,False,False,1,2,1,False,False,False,False,False,0,0.190981,0.119363,0.037135,0.005305,0.029178,0.282609,0.152174,0.108696,0.000000,0.021739,0.285714,0.142857,0.119048,0.000000,0.023810,0.282609,0.152174,0.108696,0.000000,0.021739,0.187500,0.062500,0.125000,0.000000,0.000000,0.142857,0.142857,0.000000,0.000000,0.000000
20408,8,釜元 豪,ソフトバンク,20202068,5回裏,L,楽天,False,False,False,1,2,0,False,False,False,False,False,0,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178
20410,10,加藤 翔平,ロッテ,20202536,5回裏,R,西武,False,False,False,0,2,0,False,False,False,False,False,0,0.207650,0.139344,0.038251,0.000000,0.030055,0.207650,0.139344,0.038251,0.000000,0.030055,0.207650,0.139344,0.038251,0.000000,0.030055,0.207650,0.139344,0.038251,0.000000,0.030055,0.207650,0.139344,0.038251,0.000000,0.030055,0.207650,0.139344,0.038251,0.000000,0.030055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54200,33800,ロドリゲス,オリックス,20202075,4回裏,R,日本ハム,True,False,True,1,2,0,False,False,False,False,False,0,0.191781,0.136986,0.035616,0.000000,0.019178,0.214286,0.142857,0.047619,0.000000,0.023810,0.200000,0.142857,0.028571,0.000000,0.028571,0.214286,0.142857,0.047619,0.000000,0.023810,0.333333,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
54201,33801,宮本 丈,ヤクルト,20202875,6回表,R,巨人,False,False,False,0,0,0,False,False,False,False,False,0,0.226006,0.154799,0.043344,0.000000,0.030960,0.200000,0.200000,0.000000,0.000000,0.000000,0.200000,0.200000,0.000000,0.000000,0.000000,0.200000,0.200000,0.000000,0.000000,0.000000,0.200000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
54202,33802,メヒア,西武,20202656,6回裏,L,日本ハム,False,False,False,0,0,1,False,False,False,False,False,0,0.206434,0.147453,0.032172,0.002681,0.024129,0.121212,0.090909,0.000000,0.000000,0.030303,0.200000,0.100000,0.000000,0.000000,0.100000,0.121212,0.090909,0.000000,0.000000,0.030303,0.176471,0.117647,0.000000,0.000000,0.058824,0.166667,0.166667,0.000000,0.000000,0.000000
54203,33803,坂口 智隆,ヤクルト,20202023,5回表,L,広島,False,False,False,0,0,1,False,False,False,False,False,0,0.226006,0.154799,0.043344,0.000000,0.030960,0.250000,0.200000,0.025000,0.000000,0.025000,0.375000,0.250000,0.125000,0.000000

In [37]:
FTR_btingavg[_FTR4['FTR_bt_btingavg40'].isnull()]

,id,batter,batterTeam,gameID,inning,pitcherHand,pitcherTeam,b1,b2,b3,B,O,S,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4,flg_train,FTR_bt_btingavg00,FTR_bt_btingavg01,FTR_bt_btingavg02,FTR_bt_btingavg03,FTR_bt_btingavg04,FTR_bt_btingavg10,FTR_bt_btingavg11,FTR_bt_btingavg12,FTR_bt_btingavg13,FTR_bt_btingavg14,FTR_bt_btingavg20,FTR_bt_btingavg21,FTR_bt_btingavg22,FTR_bt_btingavg23,FTR_bt_btingavg24,FTR_bt_btingavg30,FTR_bt_btingavg31,FTR_bt_btingavg32,FTR_bt_btingavg33,FTR_bt_btingavg34,FTR_bt_btingavg40,FTR_bt_btingavg41,FTR_bt_btingavg42,FTR_bt_btingavg43,FTR_bt_btingavg44,FTR_bt_btingavg50,FTR_bt_btingavg51,FTR_bt_btingavg52,FTR_bt_btingavg53,FTR_bt_btingavg54
20408,8,釜元 豪,ソフトバンク,20202068,5回裏,L,楽天,False,False,False,1,2,0,False,False,False,False,False,0,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178
20410,10,加藤 翔平,ロッテ,20202536,5回裏,R,西武,False,False,False,0,2,0,False,False,False,False,False,0,0.207650,0.139344,0.038251,0.000000,0.030055,0.207650,0.139344,0.038251,0.000000,0.030055,0.207650,0.139344,0.038251,0.000000,0.030055,0.207650,0.139344,0.038251,0.000000,0.030055,0.207650,0.139344,0.038251,0.000000,0.030055,0.207650,0.139344,0.038251,0.000000,0.030055
20411,11,太田 椋,オリックス,20202351,4回表,R,楽天,False,False,False,0,1,2,False,False,False,False,False,0,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178
20421,21,堂林 翔太,広島,20202552,8回裏,L,DeNA,True,False,True,1,1,1,False,False,False,False,False,0,0.269122,0.184136,0.036827,0.005666,0.042493,0.400000,0.266667,0.066667,0.000000,0.066667,0.222222,0.111111,0.111111,0.000000,0.000000,0.307692,0.307692,0.000000,0.000000,0.000000,0.400000,0.266667,0.066667,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000
20423,23,西野 真弘,オリックス,20202829,8回裏,R,日本ハム,False,False,False,2,1,0,False,False,False,False,False,0,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54188,33788,山下 幸輝,DeNA,20202538,2回表,R,巨人,False,False,False,2,1,1,False,False,False,False,False,0,0.269542,0.188679,0.053908,0.000000,0.026954,0.269542,0.188679,0.053908,0.000000,0.026954,0.269542,0.188679,0.053908,0.000000,0.026954,0.269542,0.188679,0.053908,0.000000,0.026954,0.269542,0.188679,0.053908,0.000000,0.026954,0.269542,0.188679,0.053908,0.000000,0.026954
54189,33789,中村 晃,ソフトバンク,20202561,3回裏,R,ロッテ,False,False,False,2,2,1,False,False,False,False,False,0,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178
54195,33795,西田 明央,ヤクルト,20202023,2回表,L,広島,False,True,False,1,0,2,False,False,False,False,False,0,0.226006,0.154799,0.043344,0.000000,0.030960,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
54196,33796,中村 晃,ソフトバンク,20202662,4回表,R,ロッテ,True,False,False,1,1,2,False,False,False,False,False,0,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.03713

In [38]:
FTR_btingavg[_FTR5['FTR_bt_btingavg50'].isnull()]

,id,batter,batterTeam,gameID,inning,pitcherHand,pitcherTeam,b1,b2,b3,B,O,S,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4,flg_train,FTR_bt_btingavg00,FTR_bt_btingavg01,FTR_bt_btingavg02,FTR_bt_btingavg03,FTR_bt_btingavg04,FTR_bt_btingavg10,FTR_bt_btingavg11,FTR_bt_btingavg12,FTR_bt_btingavg13,FTR_bt_btingavg14,FTR_bt_btingavg20,FTR_bt_btingavg21,FTR_bt_btingavg22,FTR_bt_btingavg23,FTR_bt_btingavg24,FTR_bt_btingavg30,FTR_bt_btingavg31,FTR_bt_btingavg32,FTR_bt_btingavg33,FTR_bt_btingavg34,FTR_bt_btingavg40,FTR_bt_btingavg41,FTR_bt_btingavg42,FTR_bt_btingavg43,FTR_bt_btingavg44,FTR_bt_btingavg50,FTR_bt_btingavg51,FTR_bt_btingavg52,FTR_bt_btingavg53,FTR_bt_btingavg54
20405,5,下水流 昂,楽天,20202505,4回裏,L,ロッテ,False,False,False,0,2,0,False,False,False,False,False,0,0.265625,0.177083,0.067708,0.000000,0.023438,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
20408,8,釜元 豪,ソフトバンク,20202068,5回裏,L,楽天,False,False,False,1,2,0,False,False,False,False,False,0,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178
20410,10,加藤 翔平,ロッテ,20202536,5回裏,R,西武,False,False,False,0,2,0,False,False,False,False,False,0,0.207650,0.139344,0.038251,0.000000,0.030055,0.207650,0.139344,0.038251,0.000000,0.030055,0.207650,0.139344,0.038251,0.000000,0.030055,0.207650,0.139344,0.038251,0.000000,0.030055,0.207650,0.139344,0.038251,0.000000,0.030055,0.207650,0.139344,0.038251,0.000000,0.030055
20411,11,太田 椋,オリックス,20202351,4回表,R,楽天,False,False,False,0,1,2,False,False,False,False,False,0,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178,0.191781,0.136986,0.035616,0.000000,0.019178
20416,16,小深田 大翔,楽天,20202475,6回表,L,日本ハム,False,False,False,1,0,0,False,False,False,False,False,0,0.265625,0.177083,0.067708,0.000000,0.023438,0.285714,0.285714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.200000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.285714,0.285714,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54189,33789,中村 晃,ソフトバンク,20202561,3回裏,R,ロッテ,False,False,False,2,2,1,False,False,False,False,False,0,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178
54192,33792,西 勇輝,阪神,20202516,5回表,R,広島,False,True,False,1,2,1,False,False,False,False,False,0,0.185185,0.133903,0.028490,0.002849,0.019943,0.500000,0.000000,0.250000,0.000000,0.250000,1.000000,0.000000,0.500000,0.000000,0.500000,0.500000,0.000000,0.250000,0.000000,0.250000,1.000000,0.000000,1.000000,0.000000,0.000000,0.500000,0.000000,0.250000,0.000000,0.250000
54195,33795,西田 明央,ヤクルト,20202023,2回表,L,広島,False,True,False,1,0,2,False,False,False,False,False,0,0.226006,0.154799,0.043344,0.000000,0.030960,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
54196,33796,中村 晃,ソフトバンク,20202662,4回表,R,ロッテ,True,False,False,1,1,2,False,False,False,False,False,0,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.005305,0.029178,0.190981,0.119363,0.037135,0.00

In [39]:
# 念のため欠損数の確認
FTR_btingavg.isnull().sum()

id                     0
batter                 5
batterTeam             0
gameID                 0
inning                 0
pitcherHand          116
pitcherTeam            0
b1                     0
b2                     0
b3                     0
B                      0
O                      0
S                      0
is_hit0                0
is_hit1                0
is_hit2                0
is_hit3                0
is_hit4                0
flg_train              0
FTR_bt_btingavg00      0
FTR_bt_btingavg01      0
FTR_bt_btingavg02      0
FTR_bt_btingavg03      0
FTR_bt_btingavg04      0
FTR_bt_btingavg10      0
FTR_bt_btingavg11      0
FTR_bt_btingavg12      0
FTR_bt_btingavg13      0
FTR_bt_btingavg14      0
FTR_bt_btingavg20      0
FTR_bt_btingavg21      0
FTR_bt_btingavg22      0
FTR_bt_btingavg23      0
FTR_bt_btingavg24      0
FTR_bt_btingavg30      0
FTR_bt_btingavg31      0
FTR_bt_btingavg32      0
FTR_bt_btingavg33      0
FTR_bt_btingavg34      0
FTR_bt_btingavg40      0


☝：特徴量（FTR_:）に欠損値なし

# データの保存
> ・index, id, 特徴量（FTR_:）のみ

In [40]:
cols = ['id'] + [i for i in FTR_btingavg.columns if i[:3] == 'FTR']
cols

['id',
 'FTR_bt_btingavg00',
 'FTR_bt_btingavg01',
 'FTR_bt_btingavg02',
 'FTR_bt_btingavg03',
 'FTR_bt_btingavg04',
 'FTR_bt_btingavg10',
 'FTR_bt_btingavg11',
 'FTR_bt_btingavg12',
 'FTR_bt_btingavg13',
 'FTR_bt_btingavg14',
 'FTR_bt_btingavg20',
 'FTR_bt_btingavg21',
 'FTR_bt_btingavg22',
 'FTR_bt_btingavg23',
 'FTR_bt_btingavg24',
 'FTR_bt_btingavg30',
 'FTR_bt_btingavg31',
 'FTR_bt_btingavg32',
 'FTR_bt_btingavg33',
 'FTR_bt_btingavg34',
 'FTR_bt_btingavg40',
 'FTR_bt_btingavg41',
 'FTR_bt_btingavg42',
 'FTR_bt_btingavg43',
 'FTR_bt_btingavg44',
 'FTR_bt_btingavg50',
 'FTR_bt_btingavg51',
 'FTR_bt_btingavg52',
 'FTR_bt_btingavg53',
 'FTR_bt_btingavg54']

In [41]:
FTR_btingavg[cols].to_pickle(data_path / ("%s_FTR_bt_btingavg.pkl" % prefix))